In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

d:\MyProjects\churnmodel\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
#load the model
model=load_model("model.h5")

#load the encoders
with open("GeographyEncoder.pkl","rb") as file:
    onehotencode=pickle.load(file)

with open("GenderEncoder.pkl","rb") as file:
    labelencoder=pickle.load(file)

#load the scaler
with open("Scaler.pkl","rb") as file:
    scaler=pickle.load(file)

In [3]:
#Input data

input_data={
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
#one hot encode geography
geo_encoded = onehotencode.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehotencode.get_feature_names_out(['Geography']))
geo_encoded_df

d:\MyProjects\churnmodel\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [6]:
## Encode categorical variables
input_df['Gender']=labelencoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [12]:
print(input_df["Gender"].value_counts())

Gender
Male    1
Name: count, dtype: int64


In [8]:
#Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.55012981,  0.89091075,  0.12136034, -0.68041201, -0.28051905,
         0.84584804,  0.62670381,  0.968496  , -0.91572441,  1.00400803,
        -0.57427105, -0.58350885]])

In [9]:
#prediction churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


array([[0.4873354]], dtype=float32)

In [12]:
prediction_proba=prediction[0][0]
print(prediction_proba)

0.4873354


In [13]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
